# Screening the 1000 top _Saccharomyces cerevisiae_ genbank homologues for phospharylation sites


The [ACC1 gene](http://sgd-beta.stanford.edu/locus/S000005299/overview) of _S. cerevisiae_ contains a number of phosphorylation sites. The ACC1 SGD entry has information on phosphorylated [aminoacids](https://www.yeastgenome.org/locus/S000005299/protein). [Phosphopep](http://www.sbeams.org/devDC/sbeams/cgi/Glycopeptide/peptideSearch.cgi?action=Show_hits_form;search_type=accession;organism=sce;search_term=YNR016C) and [BioGRID](https://thebiogrid.org/35841/protein) has the same information displayed in different ways.

Genbank should contain many fungal Acetyl-CoA carboxylase genes that might lack some or all of these phosphorylation sites.

The accession number of the ACC1 protein sequence is [AAA20073](http://www.ncbi.nlm.nih.gov/protein/171504/).

This CAN be done manually [here](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome&QUERY=AAA20073) like this:

![blast of AAA20073](blast_AAA20073.png)

When the blast search is finished, you will be taken to a results page that looks like this:

![blast result](blast_AAA20073_results.png)

If you scroll down the page, you can click on the links on the right side of the page for each result.
These links will give you the Genbank file for each of the sequences that produced the blast hists with your sequence like this:

![three_first_blast_results.png](three_first_blast_results.png)

However, it is very laborious. You will easily introduce errors. What if you want to redo the analysis frequently to see if new relevant sequences have been found? What if your [boss](boss.jpg) tells you he needs the same results for another protein? or the thousand best blast hits?


## Biopython

from [Wikipedia](https://en.wikipedia.org/wiki/Biopython)

"The Biopython Project is an open-source collection of non-commercial Python tools for computational biology and bioinformatics, created by an international association of developers. 

It contains classes to represent biological sequences and sequence annotations, and it is able to read and write to a variety of file formats. 

It also allows for a **programmatic means of accessing online databases of biological information**, such as those at NCBI. Separate modules extend Biopython's capabilities to sequence alignment, protein structure, population genetics, phylogenetics, sequence motifs, and machine learning.

Sounds good!

Lets look in more detail. There is an online tutorial for Biopython and it contains information on what we want [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc86).

We can use the qblast() function provided by Biopython as in the tutorial.
Qblast produces results in [XML](https://en.wikipedia.org/wiki/XML) format.

This analysis takes some time, so we would like to save the results in a file just as they did in the tutorial.

We call this file "my_blast.xml" in the code cell below:

In [2]:
Number_of_sequences = 1000

In [3]:
from Bio.Blast import NCBIWWW

result_handle = NCBIWWW.qblast("blastp", 
                               "nr", 
                               "AAA20073", 
                               hitlist_size=Number_of_sequences,
                               alignments = 1, 
                               expect=10.0)

with open("my_blast.xml", "w") as f:
    f.write(result_handle.read())    
    
result_handle.close()

The [my_blast.xml](my_blast.xml) file contains information on all High-scoring Segment Pair [HSP](https://en.wikipedia.org/wiki/BLAST#Algorithm) for each one of the most similar sequences and the parameters that were used in the analysis.

Biopython has a [parser](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc91) that can be used to extract [ACCESSION](https://en.wikipedia.org/wiki/Accession_number_%28bioinformatics%29) numbers for all the results:

In [4]:
from Bio.Blast import NCBIXML

result_handle = open("my_blast.xml")

blast_record = NCBIXML.read(result_handle)

accessions = []

for h in blast_record.alignments:
    accessions.append(h.accession)

result_handle.close()

In [5]:
len(accessions)

1000

The first five ACCESSION numbers:

In [6]:
accessions[:5]

['AAA20073', 'NP_014413', 'AJT05884', 'EDN62822', 'AJT27522']

Now we would like to get the sequences for all of the ACCESSION numbers.
This is trickier than we might expect, due to the design of the database.

We have to make a string containing all of the numbers divided by a blank:

In [7]:
query  = " ".join(accessions)
query[:60]

'AAA20073 NP_014413 AJT05884 EDN62822 AJT27522 AJT07736 AJT31'

We have to set a variable to say how many results we want. 
We call this variable retmax. We should set is to as many sequence as we have in the ids list:

In [8]:
retmax = len(accessions)

We will use functionality of the Entrez module.
We have to tell Genbank who we are when we use their service.
We can do this by setting the Entrez.email variable.

First we need the gi number associated with each accession number.

We will use the biopython wrapper for the Entrez E-utilities server programs.
Here is a tutorial from [NCBI](http://www.ncbi.nlm.nih.gov/books/NBK25501/).

[Entrez.esearch](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc112) can be used to search the E-utilities programs.

The code below posts a search on Entrez that we can fetch later

In [13]:
len(query)

10393

In [15]:
from Bio import Entrez
Entrez.email = "bjornjobb@gmail.com"

query  = " ".join(accessions)
retmax=1000
handle = Entrez.esearch( db="protein",term=query,retmax=retmax , usehistory="y")
giList = Entrez.read(handle)['IdList']

In [17]:
len(giList)

1000

In [18]:
handle = Entrez.epost(db="protein", id=",".join(giList), rettype="fasta", retmode="text")

result = Entrez.read(handle)

search_results = result

webenv, query_key  = search_results["WebEnv"], search_results["QueryKey"]

Below, we download the results in batches of 100 sequences.

In [20]:
batchSize    = 100
db="protein"


for start in range( 0, len(giList), batchSize ):

    handle = Entrez.efetch(db=db, 
                           rettype="gb", 
                           retstart=start, 
                           retmax=batchSize, 
                           webenv=webenv, 
                           query_key=query_key)

    with open("genbank_sequences.gb", "a+") as f:
        f.write(handle.read())

File with all the results (This file can be big!):

[genbank_sequences](genbank_sequences.gb)

In [21]:
# http://www.biostars.org/p/66921/
# http://www.biostars.org/p/63506/
# https://paperpile.com/shared/qlNu3j
# https://paperpile.com/shared/kdaV1s

In [29]:
from Bio import SeqIO
import re

f=open("genbank_sequences.gb","r")
sequences = SeqIO.parse(f, "gb")

seqs_wo_phos = []
seqs_w_phos = []

kinase_site  = re.compile("((M|L|F|I|V).(R|H|K)..S...(M|L|F|I|V))")
kinase_site2 = re.compile("((M|L|F|I|V)(R|H|K)...S...(M|L|F|I|V))")


for i,s in enumerate(sequences):
    ms  = list(kinase_site.finditer(str(s.seq).upper()))
    ms2 = list(kinase_site2.finditer(str(s.seq).upper()))
    if ms or ms2:
        seqs_w_phos.append(s)
    else:      
        print(i, s.id)
        seqs_wo_phos.append(s)
f.close()

557 XP_002174469.1
560 KFH45103.1
626 CDS10010.1
628 RCI05537.1
629 CEP09288.1
630 ORZ01680.1
631 CDH52590.1
661 KDE05599.1
662 SGZ26149.1
682 POY75327.1
701 KWU41533.1
705 XP_018270360.1
715 SCV70467.1
801 KDQ18913.1
886 XP_025349007.1


In [16]:
from IPython.core.display import display, HTML
for seq in seqs_wo_phos:
    display(HTML(f'<a href="https://www.ncbi.nlm.nih.gov/protein/{seq.id}">{seq.description}</a>'))

In [17]:
for seq in seqs_w_phos[0:1]+seqs_wo_phos:
    print(seq.format("fasta"))

>AAA20073.1 acetyl-CoA carboxylase [Saccharomyces cerevisiae]
MSEESLFESSPQKMEYEITNYSERHTELPGHFIGLNTVDKLEESPLRDFVKSHGGHTVIS
KILIANNGIAAVKEIRSVRKWAYETFGDDRTVQFVAMATPEDLEANAEYIRMADQYIEVP
GGTNNNNYANVDLIVDIAERADVDAVWAGWGHASENPLLPEKLSQSKRKVIFIGPPGNAM
RSLGDKISSTIVAQSAKVPCIPWSGTGVDTVHVDEKTGLVSVDDDIYQKGCCTSPEDGLQ
KAKRIGFPVMIKASEGGGGKGIRQVEREEDFIALYHQAANEIPGSPIFIMKLAGRARHLE
VQLLADQYGTNISLFGRDCSVQRRHQKIIEEAPVTIAKAETFHEMEKAAVRLGKLVGYVS
AGTVEYLYSHDDGKFYFLELNPRLQVEHPTTEMVSGVNLPAAQLQIAMGIPMHRISDIRT
LYGMNPHSASEIDFEFKTQDATKKQRRPIPKGHCTACRITSEDPNDGFKPSGGTLHELNF
RSSSNVWGYFSVGNNGNIHSFSDSQFGHIFAFGENRQASRKHMVVALKELSIRGDFRTTV
EYLIKLLETEDFEDNTITTGWLDDLITHKMTAEKPDPTLAVICGAATKAFLASEEARHKY
IESLQKGQVLSKDLLQTMFPVDFIHEGKRYKFTVAKSGNDRYTLFINGSKCDIILRQLSD
GGLLIAIGGKSHTIYWKEEVAATRLSVDSMTTLLEVENDPTQLRTPSPGKLVKFLVENGE
HIIKGQPYAEIEVMKMQMPLVSQENGIVQLLKQPGSTIVAGDIMAIMTLDDPSKVKHALP
FEGMLPDFGSPVIEGTKPAYKFKSLVSTLENILKGYDNQVIMNASLQQLIEVLRNPKLPY
SEWKLHISALHSRLPAKLDEQMEELVARSLRRGAVFPARQLSKLIDMAVKNPEYNPDKLL
GAVVEPLADIAHKYSNGLEAHEH

In [32]:
store = seqs_wo_phos
seqs_wo_phos = seqs_wo_phos[:2]

In [33]:
seqs_wo_phos

[SeqRecord(seq=Seq('MYHSCFLTERSASNVSTLFQSSSTFSTCHSPPFPPFHIFSLTSLLSQRHVRSSL...VKP', IUPACProtein()), id='XP_002174469.1', name='XP_002174469', description='acetyl-CoA/biotin carboxylase [Schizosaccharomyces japonicus yFS275]', dbxrefs=['BioProject:PRJNA32667', 'BioSample:SAMN02953668']),
 SeqRecord(seq=Seq('MPELTQNGLPHRNGKASYAEKHNIADHFIGGNRLENAPPSKVKDFVAEHDGHTV...LKD', IUPACProtein()), id='KFH45103.1', name='KFH45103', description='Acetyl-CoA carboxylase-like protein [Acremonium chrysogenum ATCC 11550]', dbxrefs=['BioProject:PRJNA248608', 'BioSample:SAMN02799700'])]

In [2]:
from Bio import Entrez

Entrez.email = "bjornjobb@gmail.com"

epost_1 = Entrez.read(Entrez.epost("protein", id=",".join(seqs_wo_phos)))

webenv = epost_1["WebEnv"]

query_key = epost_1["QueryKey"]

iden_prots = Entrez.efetch(db="protein", 
                           rettype='ipg', 
                           retmode='text', 
                           webenv=epost_1["WebEnv"], 
                           query_key=epost_1["QueryKey"])

In [3]:
# Id              Source  Nucleotide Accession    Start   Stop    Strand  Protein         Protein Name                    Organism            Strain              Assembly
# 15864892        RefSeq  NW_011627861.1          847281  854150  -       XP_002174469.1  acetyl-CoA/biotin carboxylase   Schizosaccharomyces japonicus yFS275    yFS275  GCF_000149845.2
# 15864892        RefSeq  XM_002174433.1          1       6870    +       XP_002174469.1  acetyl-CoA/biotin carboxylase   Schizosaccharomyces japonicus yFS275    yFS275  
# 15864892        INSDC   KE651167.1              847281  854150  -       EEB08176.1      acetyl-CoA/biotin carboxylase   Schizosaccharomyces japonicus yFS275    yFS275  GCA_000149845.2

In [4]:
import pandas

df = pandas.read_csv(iden_prots, sep="\t")
accno = df["Nucleotide Accession"][0]
start = df["Start"][0]
stop  = df["Stop"][0]
strand = df["Strand"][0]

from pydna.genbank import genbank

seq = genbank(accno, seq_start=start, seq_stop=stop, strand=strand)

In [5]:
seq

Gbank(NW_011627861.1 847281-854150)(-6870)

The sequences above were aligned using tcoffee (http://tcoffee.crg.cat/)

http://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi#SG12
http://article.gmane.org/gmane.comp.python.bio.general/7974/match=elink
http://article.gmane.org/gmane.comp.python.bio.general/4495/match=elink

In [6]:
cai_table = '''

Gly	GGG	17673	6.05	0.12
Gly	GGA	32723	11.20	0.23
Gly	GGT	66198	22.66	0.46
Gly	GGC	28522	9.76	0.20
	
Glu	GAG	57046	19.52	0.30
Glu	GAA	133737	45.77	0.70
Asp	GAT	111276	38.08	0.65
Asp	GAC	59389	20.33	0.35
	
Val	GTG	31266	10.70	0.19
Val	GTA	35397	12.11	0.22
Val	GTT	62735	21.47	0.39
Val	GTC	32738	11.20	0.20
	
Ala	GCG	17988	6.16	0.11
Ala	GCA	47538	16.27	0.30
Ala	GCT	59300	20.29	0.37
Ala	GCC	35410	12.12	0.22
	
Arg	AGG	27561	9.43	0.21
Arg	AGA	61537	21.06	0.48
Ser	AGT	42657	14.60	0.16
Ser	AGC	29003	9.93	0.11
	
Lys	AAG	89004	30.46	0.42
Lys	AAA	125276	42.87	0.58
Asn	AAT	107428	36.77	0.60
Asn	AAC	72290	24.74	0.40
	
Met	ATG	60426	20.68	1.00
Ile	ATA	53518	18.32	0.28
Ile	ATT	88351	30.24	0.46
Ile	ATC	49535	16.95	0.26
	
Thr	ACG	23766	8.13	0.14
Thr	ACA	53147	18.19	0.31
Thr	ACT	59096	20.23	0.34
Thr	ACC	36395	12.46	0.21
	
Trp	TGG	30179	10.33	1.00
End	TGA	1863	0.64	0.31
Cys	TGT	22879	7.83	0.62
Cys	TGC	13811	4.73	0.38
	
End	TAG	1349	0.46	0.22
End	TAA	2798	0.96	0.47
Tyr	TAT	55965	19.15	0.57
Tyr	TAC	42551	14.56	0.43
	
Leu	TTG	77261	26.44	0.28
Leu	TTA	77615	26.56	0.28
Phe	TTT	76557	26.20	0.59
Phe	TTC	52201	17.87	0.41
	
Ser	TCG	25381	8.69	0.10
Ser	TCA	55725	19.07	0.21
Ser	TCT	68207	23.34	0.26
Ser	TCC	40972	14.02	0.16
	
Arg	CGG	5299	1.81	0.04
Arg	CGA	9050	3.10	0.07
Arg	CGT	18272	6.25	0.14
Arg	CGC	7644	2.62	0.06
	
Gln	CAG	36222	12.40	0.31
Gln	CAA	79131	27.08	0.69
His	CAT	40577	13.89	0.64
His	CAC	22559	7.72	0.36
	
Leu	CTG	31054	10.63	0.11
Leu	CTA	39440	13.50	0.14
Leu	CTT	35753	12.24	0.13
Leu	CTC	16086	5.51	0.06
	
Pro	CCG	15778	5.40	0.12
Pro	CCA	51993	17.79	0.41
Pro	CCT	39685	13.58	0.31
Pro	CCC	20139	6.89	0.16'''

In [19]:
import csv

index = {}
for aa, cn, n1,n2, f in csv.reader([x for x in cai_table.splitlines() if x.strip()], delimiter='\t'):
    index[cn] = float(f)

from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex
from Bio.SeqUtils.CodonUsageIndices import SharpEcoliIndex
from pprint import pprint
cai = CodonAdaptationIndex()
cai.set_cai_index(index)

In [34]:
str(g.seq)

ATGTATCACTCGTGTTTTTTGACCGAACGTTCTGCCAGTAACGTCTCAACGCTCTTCCAATCGTCCTCTACCTTCTCTACGTGTCATTCGCCCCCTTTCCCACCATTTCACATCTTCTCTCTAACGTCTCTCCTGTCTCAACGACACGTTCGTTCCTCGTTACCACTGATACTAACCATTGCAGGAGGCAATTCCCTCAGCGTTGCCCCAGAAGGAAAGCTGAAAGACTATGTTAAGTCTCGTAATGGACACACTGTAATTTCTTCCATTCTGATCGCAAACAACGGTATTGCAGCCGTCAAGGAAATTCGAAGCATTCGGAAATGGGCTTATGAAACATTTAACGATGAAAGAGCTATTCAGTTTACGGTAATGGCCACACCTGAAGATTTGAAAGTAAACGCAGACTATATTAGAATGGCCGATCAATATATTGAGGTCCCCGGTGGCTCTAACAACAACAATTACGCAAACGTCGAACTCATTGTGGACATTGCTGAACGCATGGGAGTTCATGCCGTTTGGGCCGGTTGGGGTCACGCCTCAGAGAATCCTCGTCTGCCCGAAATGCTCGCTGCCAGTAAACAGAAAATTGTTTTTATTGGTCCTCCTGGCAATGCCATGCGTAGTTTGGGTGACAAGATTAGTTCTACAATTGTTGCTCAAAGTGCTCGCGTGCCTTGCATGCCCTGGTCCGGTAACGGCATTGACACGGTTCATGTCGACGATGACACGGGCATTGTCACAGTCGACGAAGACACATACTCAAAGGCTTGTGTGACCTCGCCCGAACAGGGTCTGCAGGTGGCCAAGGAGGTTGGCTTTCCCATTATGATTAAAGCCTCTGAAGGCGGTGGTGGTAAGGGTATCCGTAAGGTGGAGGAACCTGAAAAGTTTGCTCAGTGTTATGAACAGGTCATGGCTGAGGTTCCGGGTTCTCCTGTTTTTATCATGAAGCTTGCAGGTCATGCCCGTCATCTGGAAGTCCAGCTGCTTGCTG

In [22]:
genes = (seq,)

In [28]:
from Bio.SeqUtils import GC

for g in genes:
    print(g.id, "         ", round( cai.cai_for_gene( str(g.seq) ),3), "         ", round(GC(str(g.seq)),3))

NW_011627861.1           0.306           46.885
